 # NetVLAD Step by Step으로 구현해보기. 

 ![framework](framework.png)   

 논문의 프로젝트 페이지는 다음과 같다.   
 https://www.di.ens.fr/willow/research/netvlad/   

우선 실험에 사용한 데이터셋을 알아보자. 
- Pittsburgh 데이터셋 (Pitts250k) : contains 250k database images  downloaded from Google Street View and 24k test queries  generated from Street View but taken at different times, years apart.  
  A. Torii, J. Sivic, T. Pajdla, and M. Okutomi. Visual place recognition with repetitive structures. In Proc. CVPR, 2013.

- Tokyo 24/7 : 76k database images and 315 query images taken using mobile phone cameras.
  A. Torii, R. Arandjelović, J. Sivic, M. Okutomi, and T. Pajdla. 24/7 place recognition by view synthesis. In Proc. CVPR, 2015

- TokyoTM : Tokyo 24/7 (=test) and TokyoTM train/val are all geographically disjoint. 이건 train/Validation을 위한 셋트를 위해 추가로 사용되었다. Google 스트리트뷰 파노라마사진들이다. 

우선 피츠버그 데이터셋을 다운받아보자.   
![download](pittsburg_download.png)

In [6]:
import kagglehub
# Download latest version
path_pittsburg = kagglehub.dataset_download("duongoku/pittsburgh250k")

print("Path to dataset files:", path_pittsburg)

Path to dataset files: /home/lairpeteryksong/.cache/kagglehub/datasets/duongoku/pittsburgh250k/versions/1


데이터셋 다운로드가 이상한것 같다. 왠만해선 오리지날로 받아서 쓰자.   
http://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/

![dataset_original](dataset_origin.png)

In [17]:
import os
from requests import get
import urllib
from urllib.parse import urlparse

#https://ctkim.tistory.com/entry/%ED%8C%8C%EC%9D%B4%EC%8D%AC-urllib-%EB%AA%A8%EB%93%88#google_vignette
#https://developer0hye.tistory.com/648

url_list = []
url_list.append('https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/000.tar')
url_list.append('https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/001.tar')
url_list.append('https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/002.tar')
url_list.append('https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/003.tar')
url_list.append('https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/004.tar')
url_list.append('https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/005.tar')
url_list.append('https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/006.tar')
url_list.append('https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/007.tar')
url_list.append('https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/008.tar')
url_list.append('https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/009.tar')
url_list.append('https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/010.tar')
url_list.append('https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/groundtruth.tar')
url_list.append('https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/md5sums.txt')
url_list.append('https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/netvlad_v100_datasets.tar.gz')
url_list.append('https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/pose.txt')
url_list.append('https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/queries_real.tar')
url_list.append('https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/readGt.m')

print(url_list)
print(urlparse(url_list[0]))
print(os.path.basename(url_list[0]))

download_dir = "./data/Pittsburgh250k/"
if not os.path.exists(download_dir):
    os.mkdir(download_dir)


for url in url_list:
    print(url)
    file_name = os.path.basename(url)
    full_path = os.path.join(download_dir,file_name)
    print(full_path)
    urllib.request.urlretrieve(url,full_path)




['https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/000.tar', 'https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/001.tar', 'https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/002.tar', 'https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/003.tar', 'https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/004.tar', 'https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/005.tar', 'https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/006.tar', 'https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/007.tar', 'https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/008.tar', 'https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/009.tar', 'https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/010.tar', 'https://data.ciirc.cvut.cz/public/projects/2015netVLAD/Pittsburgh250k/groundtruth.tar', 'https://data.ciirc

In [ ]:

import tarfile

file_list = os.listdir(download_dir)
file_list_tar = [file for file in file_list if file.endswith(".tar")]

for files in file_list_tar:
    tar_folder = os.path.join(download_dir,files.split('.',1)[0])
    

    if not os.path.exists(tar_folder):
        os.mkdir(tar_folder)
    
    tar_path = os.path.join(download_dir,files)
    print(tar_path)
    print(tar_folder)

    with tarfile.open(tar_path) as tar:
        tar.extractall(path = tar_folder)
        tar.close()  



./data/Pittsburgh250k/010.tar
./data/Pittsburgh250k/010
./data/Pittsburgh250k/004.tar
./data/Pittsburgh250k/004
./data/Pittsburgh250k/008.tar
./data/Pittsburgh250k/008
./data/Pittsburgh250k/queries_real.tar
./data/Pittsburgh250k/queries_real
./data/Pittsburgh250k/000.tar
./data/Pittsburgh250k/000
./data/Pittsburgh250k/groundtruth.tar
./data/Pittsburgh250k/groundtruth
./data/Pittsburgh250k/007.tar
./data/Pittsburgh250k/007
./data/Pittsburgh250k/006.tar
./data/Pittsburgh250k/006
./data/Pittsburgh250k/002.tar
./data/Pittsburgh250k/002
./data/Pittsburgh250k/005.tar
./data/Pittsburgh250k/005
./data/Pittsburgh250k/001.tar
./data/Pittsburgh250k/001
./data/Pittsburgh250k/009.tar
./data/Pittsburgh250k/009
./data/Pittsburgh250k/003.tar
./data/Pittsburgh250k/003


## Model 다운로드

NetVLAD 에서는 Object인식을 위해 오픈된 네트워크 모델을 활용한다. 여기선 VGG-16이나 AlexNet을 사용하는데,  
이후 연구실 추천이나 다른 연구에서는 DinoV2 도 많이 활용하며, 단순히 Desciptor pooling 을 위해서는 다른 모델들도 활용하곤 한다.   
Huggingface를 이용하는 방법은 좀 더 나중에 구현해보고, 
지금은 timm 으로 받는 방법을 먼저 해본다. 

https://timm.fast.ai/create_model

In [38]:
import timm
print(timm.list_models('*vgg16*'))

['vgg16', 'vgg16_bn']


In [47]:
#임의 임력 만들기 
import torch
x = torch.randn(8,3,256,256)

#모델 선언
model = timm.create_model('vgg16', pretrained=True,features_only=True,out_indices=(4,))

result = model(x)
print(model)
print(type(result), len(result))
print(result[0].shape)

FeatureListNet(
  (features_0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (features_1): ReLU(inplace=True)
  (features_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (features_3): ReLU(inplace=True)
  (features_4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (features_5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (features_6): ReLU(inplace=True)
  (features_7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (features_8): ReLU(inplace=True)
  (features_9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (features_10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (features_11): ReLU(inplace=True)
  (features_12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (features_13): ReLU(inplace=True)
  (features_14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fe